### 6.3 히스토그램

히스토그램 : 어떠한 데이터가 많은지를 나타내는 도수 분포표를 그래프로 표현한 것

- 계급 : 일부를 묶어서 하나로 보겠다 (0-4까지를 묶어서 하나의 세트로 만듦)
- 계급 개수 : histsize, 전체를 몇 구간으로 나눌지에 대한 개수
- 계급 간격 : 나눠진 구간에 해당하는 요소의 개수
- 계급 범위 : 

In [7]:
# 6.3.1 영상 히스토그램 계산
import numpy as np, cv2

def clac_histo(image, histSize, ranges=[0, 256]):
    hist = np.zeros((histSize, 1), np.float32) # 결과가 저장되는 행렬
    gap = ranges[1] / histSize # histsize : 전체를 몇 구간으로 나눌지 개수를 가지고 있음

    for row in image:
        for pix in row:
            idx = int(pix/gap)
            hist[idx] += 1
    return hist

image = cv2.imread("images_06/pixel.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

histSize, ranges = [32], [0, 256] # 계급 개수, 값 범위 지정
gap = ranges[1]/histSize[0] # 계급 간격 계산
ranges_gap = np.arange(0, ranges[1]+1, gap) # 계급 범위

hist1 = clac_histo(image, histSize[0], ranges) # 사용자 정의 함수
hist2 = cv2.calcHist([image], [0], None, histSize, ranges) # cv 함수
hist3, bins = np.histogram(image, ranges_gap) # numpy 함수

print("사용자 정의 함수: \n", hist1.flatten())
print("open-cv 함수: \n", hist2.flatten())
print("numpy 함수: \n", hist3)

사용자 정의 함수: 
 [  97.  247.  563. 1001. 1401. 1575. 1724. 1951. 2853. 3939. 3250. 2549.
 2467. 2507. 2402. 2418. 2727. 3203. 3410. 3161. 2985. 2590. 3384. 4312.
 4764. 3489. 2802. 2238. 1127.  628.  199.   37.]
open-cv 함수: 
 [  97.  247.  563. 1001. 1401. 1575. 1724. 1951. 2853. 3939. 3250. 2549.
 2467. 2507. 2402. 2418. 2727. 3203. 3410. 3161. 2985. 2590. 3384. 4312.
 4764. 3489. 2802. 2238. 1127.  628.  199.   37.]
numpy 함수: 
 [  97  247  563 1001 1401 1575 1724 1951 2853 3939 3250 2549 2467 2507
 2402 2418 2727 3203 3410 3161 2985 2590 3384 4312 4764 3489 2802 2238
 1127  628  199   37]


In [1]:
# 6.3.1 영상 히스토그램 계산 (CV만)
import numpy as np, cv2

image = cv2.imread("images_06/pixel.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

histSize, ranges = [32], [0, 256] # 계급 개수, 값 범위 지정
gap = ranges[1]/histSize[0] # 계급 간격 계산
ranges_gap = np.arange(0, ranges[1]+1, gap) # 계급 범위

hist2 = cv2.calcHist([image], [0], None, histSize, ranges) # cv 함수

print("open-cv 함수: \n", hist2.flatten())

open-cv 함수: 
 [  97.  247.  563. 1001. 1401. 1575. 1724. 1951. 2853. 3939. 3250. 2549.
 2467. 2507. 2402. 2418. 2727. 3203. 3410. 3161. 2985. 2590. 3384. 4312.
 4764. 3489. 2802. 2238. 1127.  628.  199.   37.]


정규화 : 0-5000 범위를 0-200의 범위에 넣겠다 -> 모든 값을 25로 나누면 알아서 들어감


In [8]:
# 6.3.3 히스토그램 그래프 그리기
import numpy as np, cv2

def draw_histo(hist, shape=(200, 256)): # shape : 히스토그램 이미지 사이즈
    hist_img = np.full(shape, 255, np.uint8) # 히스토그램을 그릴 화면
    cv2.normalize(hist, hist, 0, shape[0], cv2.NORM_MINMAX) # 정규화, 최솟값이 0이고 최대값이 그래프 영상의 높이를 가지도록 조절
    # 예제에서는 -> 4764(최대)가 200이 되도록 하겠다.
    gap = hist_img.shape[1]/hist.shape[0] # 한 계급 너비

    # 빈도 값에 대한 막대 사각형을 그림
    for i, h in enumerate(hist):
        x = int(round(i * gap)) # 시작 좌표
        w = int(round(gap))
        cv2.rectangle(hist_img, (x, 0, w, int(h)), 0, cv2.FILLED)
    
    return cv2.flip(hist_img, 0) # 영상 상하 뒤집기 # 원래는 위에서부터 아래로 내려가는 히스토그램이 생성됨

image = cv2.imread("images_06/pixel.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

hist = cv2.calcHist([image], [0], None, [32], [0, 256]) # cv 함수
hist_img = draw_histo(hist)

cv2.imshow("image", image)
cv2.imshow("hist_img", hist_img)
cv2.waitKey(0)

-1

hsv 
- h : hue
- h : 채도
- v : 밝기

In [1]:
# 6.3.4 색상 히스토그램
import numpy as np, cv2

# hue 채널 팔레트 생성 (색상 히스토그램을 그리기 위해 색상을 지정하는 함수)
def make_palette(rows):
    hue = [round(i * 180 / rows) for i in range(rows)]  # hue 값 리스트 계산
    hsv = [[[h, 255, 255]] for h in hue] # (hue, 255, 255) 화소 값 계산 # gbr 아님 ㅇㅇ
    hsv = np.array(hsv, np.uint8) # 정수형 행렬 반환

    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR) # hsv -> bgr로 변경


# 히스토그램 행렬로 그래프를 그려서 반환하는 함수
def draw_hist_hue(hist, shape=(200, 256, 3)): 
    hsv_palette = make_palette(hist.shape[0]) # 색상 팔레트 생성
    hist_img = np.full(shape, 255, np.uint8) 
    cv2.normalize(hist, hist, 0, shape[0], cv2.NORM_MINMAX) # 정규화

    gap = hist_img.shape[1] / hist.shape[0] # 한 계급의 크기
    # 빈도 값에 대한 막대 사각형을 그림
    for i, h in enumerate(hist):
        x = int(round(i * gap)) # 시작 좌표
        w = int(round(gap))
        color = tuple(map(int, hsv_palette[i][0]))
        cv2.rectangle(hist_img, (x, 0, w, int(h)), color, cv2.FILLED)

    return cv2.filp(hist_img, 0)

image = cv2.imread("images_06/hue_hist.jpg", cv2.IMREAD_COLOR)
if image is None: raise Exception("영상 파일 읽기 오류")

hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV) # bgr -> hsv
hue_hist = cv2.calcHist([hsv_img], [0], None, [18], [0, 180]) # hue 채널 히스토그램 계산
hue_hist_img = draw_hist_hue(hue_hist, (200, 360, 3)) # 빈도 그래프 그리기

cv2.imshow("image", image)
cv2.imshow("hist_img", hist_img)
cv2.waitKey(0)



error: OpenCV(4.5.5) d:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0xa8565e7e::Set<3,-1,-1>,struct cv::impl::A0xa8565e7e::Set<0,5,-1>,2>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1


##### 6.3.4 히스토그램 스트래칭

특정 밝기 부분만 있는 영상(밝은/어두운 부분이 많은 영상)을 개선하는 알고리즘

가장 낮은 화소값을 0으로 당기고, 가장 높은 화소값을 255로 당김, 나머지는 비율에 맞게 조절

- 새화소값 = (화소값 - low) / (high - low) * 255

- -> 화소값 - low  -> 왼쪽으로 붙이는 작업

In [3]:
# 6.3.4 히스토그램 스트래칭
import numpy as np, cv2

def draw_histo(hist, shape=(200, 256)):
    hist_img = np.full(shape, 255, np.uint8) # 히스토그램을 그릴 화면
    cv2.normalize(hist, hist, 0, shape[0], cv2.NORM_MINMAX) # 정규화, 최솟값이 0이고 최대값이 그래프 영상의 높이를 가지도록 조절
    gap = hist_img.shape[1]/hist.shape[0] # 한 계급 너비

    # 빈도 값에 대한 막대 사각형을 그림
    for i, h in enumerate(hist):
        x = int(round(i * gap)) # 시작 좌표
        w = int(round(gap))
        cv2.rectangle(hist_img, (x, 0, w, int(h)), 0, cv2.FILLED)
    
    return cv2.flip(hist_img, 0) # 영상 상하 뒤집기 # 원래는 위에서부터 아래로 내려가는 히스토그램이 생성됨


# 빈도 값이 있는 최저 위치와 최고 위치를 찾아 반환하는 함수
def search_value_idx(hist, bias=0):
    for i in range(hist.shape[0]):
        idx = np.abs(bias - i) # 검색 위치
        if hist[idx] > 0: return idx
    return -1 # 모든 빈도 값이 0인 경우

image = cv2.imread("images_06/hist_stretch.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

bsize, ranges = [64], [0, 256] # 64 구간으로 분할(계급 개수), 화소 범위
hist = cv2.calcHist([image], [0], None, bsize, ranges) # 빈도수를 구함

bin_width = ranges[1]/bsize[0] # 한 계급의 너비
low = search_value_idx(hist, 0) * bin_width # 최저 화소값
high = search_value_idx(hist, bsize[0] - 1) * bin_width # 최고 화소값

idx = np.arange(0, 256) # 변경 시 사용할 룩업 테이블
idx = (idx - low)/(high - low) * 255 # 히스토그램 스트레칭 수식으로 룩업 테이블 값 변경

idx[0:int(low)] = 0 # 히스토그램에서 제외되는 부분 값 지정
idx[int(high+1):] = 255

dst = cv2.LUT(image, idx.astype('uint8'))

hist_dst = cv2.calcHist([dst], [0], None, bsize, ranges) # 결과 영상 히스토그램 재계산
hist_img = draw_histo(hist, (200, 360)) # 원본 영상 히스토그램
hist_dst_img = draw_histo(hist_dst, (200, 360)) # 결과 영상 히스토그램

print("high_vlue = ", high)
print("high_vlue = ", low)
cv2.imshow("image", image)
cv2.imshow("hist_img", hist_img)
cv2.imshow("dst", dst)
cv2.imshow("hist_dst_img", hist_dst_img)
cv2.waitKey(0)

    

high_vlue =  180.0
high_vlue =  52.0


-1

##### 6.3.5 히스토그램 평활화
특정 부분에서만 한쪽으로 치우친 명암 분포를 가진 영상을 균등한 분포를 갖게하는 알고리즘

-----------

평활화 과정
1. 영상의 히스토그램 계산
2. 히스토그램 빈도값에서 누적 빈도수를 계산
3. 누적 빈도수를 정규화(정규화 누적합)
4. 결과 화소값 = 정규화 누적합 * 최대 화소 값

In [4]:
# 6.3.5 히스토그램 평활화
import numpy as np, cv2

def draw_histo(hist, shape=(200, 256)):
    hist_img = np.full(shape, 255, np.uint8) # 히스토그램을 그릴 화면
    cv2.normalize(hist, hist, 0, shape[0], cv2.NORM_MINMAX) # 정규화, 최솟값이 0이고 최대값이 그래프 영상의 높이를 가지도록 조절
    gap = hist_img.shape[1]/hist.shape[0] # 한 계급 너비

    # 빈도 값에 대한 막대 사각형을 그림
    for i, h in enumerate(hist):
        x = int(round(i * gap)) # 시작 좌표
        w = int(round(gap))
        cv2.rectangle(hist_img, (x, 0, w, int(h)), 0, cv2.FILLED)
    
    return cv2.flip(hist_img, 0) # 영상 상하 뒤집기 # 원래는 위에서부터 아래로 내려가는 히스토그램이 생성됨


image = cv2.imread("images_06/equalize.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

bins, ranges = [256], [0, 256] 
hist = cv2.calcHist([image], [0], None, bins, ranges)

# 히스토그램 누적합 계산
accum_hist = np.zeros(hist.shape[:2], np.float32)
accum_hist[0] = hist[0]
for i in range(1, hist.shape[0]):
    accum_hist[i] = accum_hist[i - 1] + hist[i]

accum_hist = (accum_hist / sum(hist)) * 255 # 누적합의 정규화
dst1 = [[accum_hist[val] for val in row] for row in image] # 화소 값 할당
dst1 = np.array(dst1, np.uint8) 

dst2 = cv2.equalizeHist(image)
hist1 = cv2.calcHist([dst1], [0], None, bins, ranges)
hist2 = cv2.calcHist([dst2], [0], None, bins, ranges)
hist_img = draw_histo(hist)
hist_img1 = draw_histo(hist1)
hist_img2 = draw_histo(hist2)

cv2.imshow("image", image)
cv2.imshow("hist_img", hist_img)
cv2.imshow("dst1", dst1)
cv2.imshow("dst2", dst2)
cv2.imshow("hist_img", hist_img)
cv2.imshow("hist_img2", hist_img2)
cv2.imshow("hist_img1", hist_img1)
cv2.waitKey(0)





-1